## 清理

In [111]:
import os
import json
import shutil
import re

def clean_log_directory(log_dir):
    if not os.path.isdir(log_dir):
        print(f"目录 {log_dir} 不存在！")
        return

    for sub_dir in os.listdir(log_dir):
        sub_dir_path = os.path.join(log_dir, sub_dir)
        r"best_model_([^\W_]+)_(\w+).pth$"
        match = re.search(r"\d+-\d+_\d+:\d+:\d+_(\w+)_([^\W_]+)_(\d+)", sub_dir)
        model_name = None
        if match:
            model_name = match.group(1)
            method = match.group(2)
            win_len = match.group(3)
        else:
            continue
        if os.path.isdir(sub_dir_path):
            found_pth = False
            record_model_name = None
            for file in os.listdir(sub_dir_path):
                if file.endswith(".pth"):
                    found_pth = True
                    # match = re.search(r"best_model_([^\W_]+)_(\w+).pth$", file)
                    # if match:
                    #     record_model_name = match.group(1)
                    #     if 'attention' in model_name and 'attention' in record_model_name.lower():
                    #         found_pth = True
                    #         break
                    #     if 'crnn' in model_name and 'crnn' in record_model_name.lower():
                    #         found_pth = True
                    #         break
                    #     if 'resnet' in model_name and 'resnet' in record_model_name.lower():
                    #         found_pth = True
                    #         break
                    #     if 'lstm' in model_name and 'lstm' in record_model_name.lower():
                    #         found_pth = True
                    #         break
                    #     if 'cnn' in model_name and 'simplecnn' in record_model_name.lower():
                    #         found_pth = True
                    #         break
            if not found_pth:
                print(f"删除子目录: {sub_dir_path}, {record_model_name}")
                shutil.rmtree(sub_dir_path)

clean_log_directory('checkpoints')

删除子目录: checkpoints/12-22_12:40:37_attention_cnn_stft_derivatives_2048, None
删除子目录: checkpoints/12-22_12:31:24_crnn_stft_derivatives_2048, None
删除子目录: checkpoints/12-22_12:38:01_attention_cnn_stft_2048, None
删除子目录: checkpoints/12-22_12:21:10_attention_cnn_stft_2048, None
删除子目录: checkpoints/12-22_12:16:04_crnn_stft_derivatives_2048, None
删除子目录: checkpoints/12-22_12:24:04_attention_cnn_stft_derivatives_2048, None


In [34]:
sub_dir='12-22_04:35:12_attention_cnn_fft_1024'
re.search(r"\d+-\d+_\d+:\d+:\d+_(\w+)_([^\W_]+)_(\d+)", sub_dir)

<re.Match object; span=(0, 37), match='12-22_04:35:12_attention_cnn_fft_1024'>

# 日志抓取

In [221]:
import os
import json
import re


def generate_commands(experiments, batch_size=64):
    """
    Generate command line arguments for each experiment configuration.

    Args:
        experiments: List of tuples containing (model, window_length, method, transform)
        batch_size: Default batch size for training

    Returns:
        List of command lists ready for subprocess execution
    """
    commands = []

    for model, window_length, method, transform in experiments:
        if model == 'lstm' or method in ['raw', 'fft']:
            continue
        cmd = ["python", "main.py"]
        cmd.extend(["--model", model])
        cmd.extend(["--window_length", window_length])
        cmd.extend(["--method", method])
        cmd.extend(["--transform", transform])
        if method == "stft_derivatives":
            cmd.extend(["--batch_size", str(40)])
        else:
            cmd.extend(["--batch_size", str(batch_size)])
        commands.append(cmd)

    return commands


def extract_log_directory(log_dir, clean_func=None):
    if not os.path.isdir(log_dir):
        print(f"目录 {log_dir} 不存在！")
        return
    if clean_func:
        clean_func(log_dir)

    results = []
    for sub_dir in os.listdir(log_dir):
        sub_dir_path = os.path.join(log_dir, sub_dir)

        if os.path.isdir(sub_dir_path):
            results.append(extract_summary(sub_dir_path))
    return results


def extract_summary(log_dir):
    # 确保 log_dir 是存在的
    if not os.path.isdir(log_dir):
        print(f"目录 {log_dir} 不存在！")
        return
    retrieval_res = None
    acc = 0
    config = {}
    model_name = ""
    method = ""
    for file in os.listdir(log_dir):
        if file == "evaluation_results.json":
            with open(os.path.join(log_dir, file), "r") as f:
                results = json.load(f)
            retrieval_res = results["result"]
            config.update(results["config"])
        if file == "config.json":
            with open(os.path.join(log_dir, file), "r") as f:
                cfg = json.load(f)
            config.update(cfg)
    for file in os.listdir(log_dir):
        method = config["method"]
        pattern = f"{method}" + r"_\d+\.\d?"
        match = re.search(pattern, file)
        if match:
            acc = float(file.split("_")[-1])
            break

    return {
        "config": config,
        "retrieval_res": retrieval_res,
        "acc": acc,
        "path": log_dir,
    }


def extract_retr(log_dir):
    # 确保 log_dir 是存在的
    if not os.path.isdir(log_dir):
        print(f"目录 {log_dir} 不存在！")
        return
    retrieval_res = None
    acc = 0
    config = {}
    method = ""
    for file in os.listdir(log_dir):
        if file == "evaluation_results.json":
            with open(os.path.join(log_dir, file), "r") as f:
                results = json.load(f)
            retrieval_res = results["result"]
            config.update(results["config"])
        if file == "config.json":
            with open(os.path.join(log_dir, file), "r") as f:
                cfg = json.load(f)
            config.update(cfg)
    for file in os.listdir(log_dir):
        method = config["method"]
        pattern = f"{method}" + r"_\d+\.\d?"
        match = re.search(pattern, file)
        if match:
            acc = float(file.split("_")[-1])
            break

    return {
        "config": config,
        "retrieval_res": retrieval_res,
        "acc": acc,
        "path": log_dir,
    }

    if not os.path.isdir(log_dir):
        print(f"目录 {log_dir} 不存在！")
        return

In [222]:
res = extract_log_directory('checkpoints')

In [234]:
import itertools
run_set = set()
models = ["cnn", "crnn", "attention_cnn", "lstm", "resnet"]
window_lengths = ["1024", "2048", "4096"]

configurations = [
    {"method": "raw", "batch_size": "256"},
    {"method": "fft", "batch_size": "200"},
    {"method": "mel", "batch_size": "1500"},
    {"method": "mfcc", "batch_size": "100", "transform": "dst"},
    {"method": "mfcc", "batch_size": "100", "transform": "dct"},
    {"method": "mfcc_derivatives", "batch_size": "100", "transform": "dst"},
    {"method": "mfcc_derivatives", "batch_size": "100", "transform": "dct"},
    {"method": "stft", "batch_size": "100"},
    {"method": "stft_DCT", "batch_size": "100"},
    {"method": "stft_derivatives", "batch_size": "100"},
]

for model, win_len, cfg in itertools.product(models, window_lengths, configurations):
    run_set.add(
        ('--model', model, '--window_length', win_len, '--method', cfg["method"], '--batch_size', cfg["batch_size"], '--transform', cfg.get("transform", "dct"))
    )

In [235]:
len(res)

100

In [236]:
succe_run = {}
out_dir = []
for r in res:
    if r["retrieval_res"] is None:
        continue
    model = r["config"]["model"]
    win_len = r["config"]["window_length"]
    cfg = r["config"]

    succe_run[(model, str(win_len), cfg["method"], cfg.get("transform", "dct"))] = r

In [237]:
acc_pair = []
retr_pair = []
for k,v in succe_run.items():
    acc_pair.append((k, v['acc']))
    retr_pair.append((k, v['retrieval_res']))
acc_pair.sort(key=lambda x: x[0], reverse=True)
retr_pair.sort(key=lambda x: x[0], reverse=True)
acc_pair

[(('resnet', '4096', 'stft_derivatives', 'dct'), 50.5),
 (('resnet', '4096', 'stft_DCT', 'dct'), 41.25),
 (('resnet', '4096', 'stft', 'dct'), 60.0),
 (('resnet', '4096', 'mfcc_derivatives', 'dst'), 63.0),
 (('resnet', '4096', 'mfcc_derivatives', 'dct'), 62.75),
 (('resnet', '4096', 'mfcc', 'dst'), 55.5),
 (('resnet', '4096', 'mfcc', 'dct'), 54.75),
 (('resnet', '4096', 'mel', 'dct'), 68.25),
 (('resnet', '2048', 'stft_derivatives', 'dct'), 65.5),
 (('resnet', '2048', 'stft_DCT', 'dct'), 50.75),
 (('resnet', '2048', 'stft', 'dct'), 67.5),
 (('resnet', '2048', 'raw', 'dct'), 42.25),
 (('resnet', '2048', 'mfcc_derivatives', 'dst'), 68.25),
 (('resnet', '2048', 'mfcc_derivatives', 'dct'), 65.75),
 (('resnet', '2048', 'mfcc', 'dst'), 55.25),
 (('resnet', '2048', 'mfcc', 'dct'), 57.0),
 (('resnet', '2048', 'mel', 'dct'), 70.5),
 (('resnet', '1024', 'stft_derivatives', 'dct'), 74.0),
 (('resnet', '1024', 'stft_DCT', 'dct'), 52.75),
 (('resnet', '1024', 'stft', 'dct'), 75.75),
 (('resnet', '10

In [238]:
retr_pair

[(('resnet', '4096', 'stft_derivatives', 'dct'),
  {'MRR@10': 0.5521438492063492,
   'NDCG@10': 0.47074955592293594,
   'Binary Metric@10': 0.7675,
   'Proportion Metric@10': 0.373,
   'MRR@20': 0.5572193782417273,
   'NDCG@20': 0.6044850176975618,
   'Binary Metric@20': 0.8375,
   'Proportion Metric@20': 0.33375}),
 (('resnet', '4096', 'stft_DCT', 'dct'),
  {'MRR@10': 0.45614484126984123,
   'NDCG@10': 0.41766124488033896,
   'Binary Metric@10': 0.6225,
   'Proportion Metric@10': 0.36225,
   'MRR@20': 0.46439121768968905,
   'NDCG@20': 0.5233157997184725,
   'Binary Metric@20': 0.74,
   'Proportion Metric@20': 0.338375}),
 (('resnet', '4096', 'stft', 'dct'),
  {'MRR@10': 0.6235257936507935,
   'NDCG@10': 0.5394207747916139,
   'Binary Metric@10': 0.7875,
   'Proportion Metric@10': 0.46299999999999997,
   'MRR@20': 0.6284984337987435,
   'NDCG@20': 0.6611329847732644,
   'Binary Metric@20': 0.86,
   'Proportion Metric@20': 0.42075}),
 (('resnet', '4096', 'mfcc_derivatives', 'dst'),
  {

In [239]:
len(acc_pair)

88

## 比较各个模型，固定window_size，比较DCT 和 DST

In [240]:
import itertools
model_list = ['cnn', 'crnn', 'attention_cnn', 'lstm', 'resnet']
method = [
    ('mel', 'dct'),
    ('stft', 'dct'),
    ('stft_DCT', 'dct'),
    ('stft_derivatives', 'dct'),
    ('mfcc', 'dst'),
    ('mfcc', 'dct'),
    ('mfcc_derivatives', 'dst'),
    ('mfcc_derivatives', 'dct'),
]
needed = {
    (model,"2048", *method): None
    for model, method in itertools.product(model_list, method)
}
for k ,acc in acc_pair:
    if k in needed:
        needed[k] = acc

In [241]:
cmd1 = generate_commands([k for k, v in needed.items() if v is None])
cmd1

[['python',
  'main.py',
  '--model',
  'crnn',
  '--window_length',
  '2048',
  '--method',
  'stft',
  '--transform',
  'dct',
  '--batch_size',
  '64'],
 ['python',
  'main.py',
  '--model',
  'crnn',
  '--window_length',
  '2048',
  '--method',
  'stft_derivatives',
  '--transform',
  'dct',
  '--batch_size',
  '40'],
 ['python',
  'main.py',
  '--model',
  'attention_cnn',
  '--window_length',
  '2048',
  '--method',
  'stft',
  '--transform',
  'dct',
  '--batch_size',
  '64'],
 ['python',
  'main.py',
  '--model',
  'attention_cnn',
  '--window_length',
  '2048',
  '--method',
  'stft_derivatives',
  '--transform',
  'dct',
  '--batch_size',
  '40']]

In [242]:
needed

{('cnn', '2048', 'mel', 'dct'): 38.5,
 ('cnn', '2048', 'stft', 'dct'): 15.75,
 ('cnn', '2048', 'stft_DCT', 'dct'): 27.5,
 ('cnn', '2048', 'stft_derivatives', 'dct'): 10.75,
 ('cnn', '2048', 'mfcc', 'dst'): 37.5,
 ('cnn', '2048', 'mfcc', 'dct'): 34.5,
 ('cnn', '2048', 'mfcc_derivatives', 'dst'): 30.25,
 ('cnn', '2048', 'mfcc_derivatives', 'dct'): 32.0,
 ('crnn', '2048', 'mel', 'dct'): 23.0,
 ('crnn', '2048', 'stft', 'dct'): None,
 ('crnn', '2048', 'stft_DCT', 'dct'): 18.25,
 ('crnn', '2048', 'stft_derivatives', 'dct'): None,
 ('crnn', '2048', 'mfcc', 'dst'): 18.75,
 ('crnn', '2048', 'mfcc', 'dct'): 19.0,
 ('crnn', '2048', 'mfcc_derivatives', 'dst'): 13.5,
 ('crnn', '2048', 'mfcc_derivatives', 'dct'): 13.75,
 ('attention_cnn', '2048', 'mel', 'dct'): 51.0,
 ('attention_cnn', '2048', 'stft', 'dct'): None,
 ('attention_cnn', '2048', 'stft_DCT', 'dct'): 31.0,
 ('attention_cnn', '2048', 'stft_derivatives', 'dct'): None,
 ('attention_cnn', '2048', 'mfcc', 'dst'): 37.25,
 ('attention_cnn', '204

In [243]:
import pandas as pd

# Create empty DataFrame with desired index and columns
models = ["resnet", "CNN", "LSTM", "CRNN", "AttentionCNN"]
features = [
    "mel",
    "stft",
    "stft_DCT",
    "stft_derivatives",
    "mfcc_dst",
    "mfcc_dct",
    "mfcc_derivatives_dst",
    "mfcc_derivatives_dct",
]

df = pd.DataFrame(index=models, columns=features)

# Fill DataFrame with values from dictionary
for (model, window, feature, transform), value in needed.items():
    # Convert model name to match desired format
    model_map = {
        "cnn": "CNN",
        "lstm": "LSTM",
        "crnn": "CRNN",
        "attention_cnn": "AttentionCNN",
        "resnet": "resnet",
    }

    # Map feature types to column names
    feature_map = {
        "mel": "mel",
        "mfcc_derivatives": "mfcc_derivatives_dst"
        if transform == "dst"
        else "mfcc_derivatives_dct",
        "stft": "stft",
        "stft_DCT": "stft_DCT",
        "stft_derivatives": "stft_derivatives",
        "mfcc": "mfcc_dst" if transform == "dst" else "mfcc_dct",
    }

    model_name = model_map[model]
    feature_name = feature_map.get(feature, feature)

    if feature_name in features:  # Only add if feature is in our columns list
        df.at[model_name, feature_name] = value

# Save to CSV
df.to_csv("results.csv")


In [233]:
df

,mel,stft,stft_DCT,stft_derivatives,mfcc_dst,mfcc_dct,mfcc_derivatives_dst,mfcc_derivatives_dct
resnet,70.5,67.5,50.75,65.5,55.25,57.0,68.25,65.75
CNN,38.5,15.75,27.5,10.75,37.5,34.5,30.25,32.0
LSTM,None,None,None,None,6.5,6.25,None,None
CRNN,23.0,None,18.25,None,18.75,19.0,13.5,13.75
AttentionCNN,51.0,None,31.0,None,37.25,40.5,33.0,37.75


## 比较窗口大小

In [196]:
import itertools

window_lengths = ["1024", "2048", "4096"]
method = [
    ("raw", "dct"),
    ("mel", "dct"),
    ("mfcc", "dst"),
    ("mfcc", "dct"),
    ("mfcc_derivatives", "dst"),
    ("mfcc_derivatives", "dct"),
    ("stft", "dct"),
    ("stft_DCT", "dct"),
    ("stft_derivatives", "dct"),
]
needed = {
    ("resnet", wid, *method): None
    for wid, method in itertools.product(window_lengths, method)
}
for k, acc in acc_pair:
    if k in needed:
        needed[k] = acc

In [197]:
cmd2 = generate_commands([k for k, v in needed.items() if v is None])
cmd2

[['python',
  'main.py',
  '--model',
  'resnet',
  '--window_length',
  '4096',
  '--method',
  'stft',
  '--transform',
  'dct',
  '--batch_size',
  '64'],
 ['python',
  'main.py',
  '--model',
  'resnet',
  '--window_length',
  '4096',
  '--method',
  'stft_derivatives',
  '--transform',
  'dct',
  '--batch_size',
  '40']]

In [220]:
import pandas as pd

# 定义所需的方法和指标
methods = [
    "raw_dct",
    "mel_dct",
    "mfcc_dst",
    "mfcc_dct", 
    "mfcc_derivatives_dst",
    "mfcc_derivatives_dct",
    "stft_dct",
    "stft_DCT_dct",
    "stft_derivatives_dct"
]

metrics = [
    "MRR@10",
    "MRR@20",
    "NDCG@10", 
    "NDCG@20",
    "Binary Metric@10",
    "Binary Metric@20",
    "Proportion Metric@10",
    "Proportion Metric@20"
]

# 创建空的DataFrame
df = pd.DataFrame(index=methods, columns=metrics)

# 创建特征映射字典
feature_map = {
    ('raw', 'dct'): 'raw_dct',
    ('mel', 'dct'): 'mel_dct',
    ('mfcc', 'dst'): 'mfcc_dst',
    ('mfcc', 'dct'): 'mfcc_dct',
    ('mfcc_derivatives', 'dst'): 'mfcc_derivatives_dst',
    ('mfcc_derivatives', 'dct'): 'mfcc_derivatives_dct',
    ('stft', 'dct'): 'stft_dct',
    ('stft_DCT', 'dct'): 'stft_DCT_dct',
    ('stft_derivatives', 'dct'): 'stft_derivatives_dct'
}

# 填充DataFrame
for (model, window, feature, transform), values in needed.items():
    
    method_name = feature_map.get((feature, transform))
    if method_name in methods:
        for metric, value in values.items():
            if metric in metrics:
                df.at[method_name, metric] = round(value, 3)

# 保存到CSV
df.to_csv("metrics_results.csv")

In [199]:
df

,mel,stft,stft_DCT,stft_derivatives,mfcc_dst,mfcc_dct,mfcc_derivatives_dst,mfcc_derivatives_dct
1024,68.75,75.75,52.75,74.0,61.5,61.5,66.0,67.5
2048,70.5,67.5,50.75,65.5,55.25,57.0,68.25,65.75
4096,68.25,None,41.25,None,55.5,54.75,63.0,62.75


In [200]:
(cmd1 + cmd2)

[['python',
  'main.py',
  '--model',
  'crnn',
  '--window_length',
  '2048',
  '--method',
  'stft',
  '--transform',
  'dct',
  '--batch_size',
  '64'],
 ['python',
  'main.py',
  '--model',
  'crnn',
  '--window_length',
  '2048',
  '--method',
  'stft_derivatives',
  '--transform',
  'dct',
  '--batch_size',
  '40'],
 ['python',
  'main.py',
  '--model',
  'attention_cnn',
  '--window_length',
  '2048',
  '--method',
  'stft',
  '--transform',
  'dct',
  '--batch_size',
  '64'],
 ['python',
  'main.py',
  '--model',
  'attention_cnn',
  '--window_length',
  '2048',
  '--method',
  'stft_derivatives',
  '--transform',
  'dct',
  '--batch_size',
  '40'],
 ['python',
  'main.py',
  '--model',
  'resnet',
  '--window_length',
  '4096',
  '--method',
  'stft',
  '--transform',
  'dct',
  '--batch_size',
  '64'],
 ['python',
  'main.py',
  '--model',
  'resnet',
  '--window_length',
  '4096',
  '--method',
  'stft_derivatives',
  '--transform',
  'dct',
  '--batch_size',
  '40']]

## 检索指标收集

In [216]:
retr_pair

[(('resnet', '4096', 'stft_DCT', 'dct'),
  {'MRR@10': 0.45614484126984123,
   'NDCG@10': 0.41766124488033896,
   'Binary Metric@10': 0.6225,
   'Proportion Metric@10': 0.36225,
   'MRR@20': 0.46439121768968905,
   'NDCG@20': 0.5233157997184725,
   'Binary Metric@20': 0.74,
   'Proportion Metric@20': 0.338375}),
 (('resnet', '4096', 'mfcc_derivatives', 'dst'),
  {'MRR@10': 0.4210674603174603,
   'NDCG@10': 0.38115059804468243,
   'Binary Metric@10': 0.7125,
   'Proportion Metric@10': 0.2115,
   'MRR@20': 0.4277403072288908,
   'NDCG@20': 0.48935858842732655,
   'Binary Metric@20': 0.8075,
   'Proportion Metric@20': 0.16575000000000004}),
 (('resnet', '4096', 'mfcc_derivatives', 'dct'),
  {'MRR@10': 0.37734226190476194,
   'NDCG@10': 0.33711235137538526,
   'Binary Metric@10': 0.6375,
   'Proportion Metric@10': 0.17825,
   'MRR@20': 0.3875925836182222,
   'NDCG@20': 0.4545786737046384,
   'Binary Metric@20': 0.775,
   'Proportion Metric@20': 0.1475}),
 (('resnet', '4096', 'mfcc', 'dst'),

In [217]:
import itertools

metrics = [
    "MRR@10",
    "MRR@20",
    "NDCG@10",
    "NDCG@20",
    "Binary Metric@10",
    "Binary Metric@20",
    "Proportion Metric@10",
    "Proportion Metric@20",
]
method = [
    ("raw", "dct"),
    ("mel", "dct"),
    ("mfcc", "dst"),
    ("mfcc", "dct"),
    ("mfcc_derivatives", "dst"),
    ("mfcc_derivatives", "dct"),
    ("stft", "dct"),
    ("stft_DCT", "dct"),
    ("stft_derivatives", "dct"),
]
needed = {("resnet", "2048", *m): None for m in method}
for k, v in retr_pair:
    if k in needed:
        needed[k] = v

In [218]:
needed

{('resnet', '2048', 'raw', 'dct'): {'MRR@10': 0.4349156746031746,
  'NDCG@10': 0.3673604132948862,
  'Binary Metric@10': 0.7,
  'Proportion Metric@10': 0.2545,
  'MRR@20': 0.441815859428669,
  'NDCG@20': 0.5029928143955349,
  'Binary Metric@20': 0.8,
  'Proportion Metric@20': 0.224375},
 ('resnet', '2048', 'mel', 'dct'): {'MRR@10': 0.433844246031746,
  'NDCG@10': 0.38726597538404256,
  'Binary Metric@10': 0.7025,
  'Proportion Metric@10': 0.21525,
  'MRR@20': 0.4428025971061962,
  'NDCG@20': 0.5059710117048668,
  'Binary Metric@20': 0.83,
  'Proportion Metric@20': 0.17162500000000003},
 ('resnet', '2048', 'mfcc', 'dst'): {'MRR@10': 0.3914325396825397,
  'NDCG@10': 0.36097735812890674,
  'Binary Metric@10': 0.7025,
  'Proportion Metric@10': 0.18000000000000005,
  'MRR@20': 0.4001211025386722,
  'NDCG@20': 0.4774021532069592,
  'Binary Metric@20': 0.8275,
  'Proportion Metric@20': 0.14700000000000002},
 ('resnet', '2048', 'mfcc', 'dct'): {'MRR@10': 0.37964087301587296,
  'NDCG@10': 0.339

In [ ]:
import pandas as pd

# Create empty DataFrame with desired index and columns
metrics = [
    "MRR@10",
    "MRR@20",
    "NDCG@10",
    "NDCG@20",
    "Binary Metric@10",
    "Binary Metric@20",
    "Proportion Metric@10",
    "Proportion Metric@20",
]
features = [
    "mel",
    "stft",
    "stft_DCT",
    "stft_derivatives",
    "mfcc_dst",
    "mfcc_dct",
    "mfcc_derivatives_dst",
    "mfcc_derivatives_dct",
]
df = pd.DataFrame(index=feature, columns=metrics)

# Fill DataFrame with values from dictionary
for (model, window, feature, transform), value in needed.items():
    # Convert model name to match desired format
    model_map = {
        "cnn": "CNN",
        "lstm": "LSTM",
        "crnn": "CRNN",
        "attention_cnn": "AttentionCNN",
        "resnet": "resnet",
    }

    # Map feature types to column names
    feature_map = {
        "mel": "mel",
        "mfcc_derivatives": "mfcc_derivatives_dst"
        if transform == "dst"
        else "mfcc_derivatives_dct",
        "stft": "stft",
        "stft_DCT": "stft_DCT",
        "stft_derivatives": "stft_derivatives",
        "mfcc": "mfcc_dst" if transform == "dst" else "mfcc_dct",
    }

    # window = model_map[model]
    feature_name = feature_map.get(feature, feature)

    if feature_name in features:  # Only add if feature is in our columns list
        df.at[window, feature_name] = value

# Save to CSV
df.to_csv("results.csv")


# 检索任务

## 需要重新跑的任务

In [100]:
import os
import json
import re


def extract_log_directory(log_dir, clean_func=None):
    if not os.path.isdir(log_dir):
        print(f"目录 {log_dir} 不存在！")
        return
    if clean_func:
        clean_func(log_dir)

    results = []
    for sub_dir in os.listdir(log_dir):
        sub_dir_path = os.path.join(log_dir, sub_dir)

        if os.path.isdir(sub_dir_path):
            if not enough_rec(sub_dir_path):
                results.append(sub_dir_path)
                continue
    return results


def extract_summary(log_dir):
    # 确保 log_dir 是存在的
    if not os.path.isdir(log_dir):
        print(f"目录 {log_dir} 不存在！")
        return
    retrieval_res = None
    acc = 0
    config = {}
    method = ""
    for file in os.listdir(log_dir):
        if file == "evaluation_results.json":
            with open(os.path.join(log_dir, file), "r") as f:
                results = json.load(f)
            retrieval_res = results["result"]
            config.update(results["config"])
        if file == "config.json":
            with open(os.path.join(log_dir, file), "r") as f:
                cfg = json.load(f)
            config.update(cfg)

    # for file in os.listdir(log_dir):
    # method = config["method"]
    # pattern = f"{method}" + r"_\d+\.\d?"
    # match = re.search(pattern, file)
    # if match:
    #     acc = float(file.split("_")[-1])
    #     break

    return {
        "config": config,
        "retrieval_res": retrieval_res,
        # "acc": acc,
        "path": log_dir,
    }


def enough_rec(log_dir):
    # 确保 log_dir 是存在的
    if not os.path.isdir(log_dir):
        print(f"目录 {log_dir} 不存在！")
        return
    if 'lstm' in log_dir:
        return True
    for file in os.listdir(log_dir):
        if file == "retrieval_results.jsonl":
            with open(os.path.join(log_dir, file), "r") as f:
                for line in f:
                    qa = json.loads(line)
                    if len(list(qa.values())[0]) < 20:
                        return False
                    else:
                        return True
    return True


In [101]:
not_enough_rec = extract_log_directory('checkpoints')

In [102]:
not_enough_rec

['checkpoints/12-22_05:37:25_cnn_mfcc_derivatives_1024',
 'checkpoints/12-22_05:47:41_resnet_mfcc_derivatives_2048',
 'checkpoints/12-22_06:01:03_cnn_stft_2048',
 'checkpoints/12-22_04:36:59_resnet_fft_1024',
 'checkpoints/12-22_05:58:43_resnet_mfcc_derivatives_4096',
 'checkpoints/12-22_05:48:59_cnn_mfcc_derivatives_1024',
 'checkpoints/12-22_05:58:33_resnet_mfcc_derivatives_2048']

## 只需要重新metric

In [103]:
import os
import json
import re


def extract_log_directory(log_dir, clean_func=None):
    if not os.path.isdir(log_dir):
        print(f"目录 {log_dir} 不存在！")
        return
    if clean_func:
        clean_func(log_dir)

    results = []
    for sub_dir in os.listdir(log_dir):
        sub_dir_path = os.path.join(log_dir, sub_dir)

        if os.path.isdir(sub_dir_path):
            if not enough_rec(sub_dir_path):
                results.append(sub_dir_path)
                continue
    return results


def extract_summary(log_dir):
    # 确保 log_dir 是存在的
    if not os.path.isdir(log_dir):
        print(f"目录 {log_dir} 不存在！")
        return
    retrieval_res = None
    config = {}
    for file in os.listdir(log_dir):
        if file == "evaluation_results.json":
            with open(os.path.join(log_dir, file), "r") as f:
                results = json.load(f)
            retrieval_res = results["result"]
            config.update(results["config"])
        if file == "config.json":
            with open(os.path.join(log_dir, file), "r") as f:
                cfg = json.load(f)
            config.update(cfg)

    return {
        "config": config,
        "retrieval_res": retrieval_res,
        # "acc": acc,
        "path": log_dir,
    }


def enough_data(log_dir):
    # 确保 log_dir 是存在的
    if not os.path.isdir(log_dir):
        print(f"目录 {log_dir} 不存在！")
        return
    for file in os.listdir(log_dir):
        if file == "retrieval_results.jsonl":
            with open(os.path.join(log_dir, file), "r") as f:
                for line in f:
                    qa = json.loads(line)
                    if len(list(qa.values())[0]) >= 20:
                        return True
    return False